### 7-3 데이터변형. 180313
> p272~282: 지각해서 놓친 부분

---
1. p272 중복제거
2. p273 함수/매핑으로 데이터 변형
3. p275 값치환
4. p277 축색인 이름바꾸기
5. p279 개별화 양자화
6. p282 특이값 찾기/제외
7. p284 치환, 임의 샘플링
8. p285 표시자/더비 변수

## 중복제거
p272

In [2]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [3]:
data= DataFrame({'k1':['one']*3+ ['two']*4,
                'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [4]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [5]:
data[data.duplicated()]

,k1,k2
1,one,1
4,two,3
6,two,4


In [6]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [9]:
data['v1']= range(7); data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [15]:
data[data.duplicated()]

,k1,k2,v1


In [13]:
data[data.duplicated(['k1'])]

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
5,two,4,5
6,two,4,6


In [19]:
#얕은 복사
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


In [20]:
data[data.duplicated(['k1','k2'])]

,k1,k2,v1
1,one,1,1
4,two,3,4
6,two,4,6


##### keep= 'last'
마지막으로 발견된 값을 반환?
같은거1, 같은거2 있으면 같은거 2를 남김

In [23]:
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [25]:
#one 1 0, one 1 1 중에 one 1 0을 남겼다
data.drop_duplicates(['k1','k2'])

,k1,k2,v1
0,one,1,0
2,one,2,2
3,two,3,3
5,two,4,5


In [21]:
#one 1 0, one 1 1 중에 one 1 1 을 남겼다
data.drop_duplicates(['k1','k2'], keep= 'last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


## 함수/매핑으로 변형
p273

In [163]:
data= DataFrame({'food':['bacon', 'pulled pork', 'bacon', 'Pastrami',
                        'corned beef', 'Bacon', 'pastrami', 
                        'honey ham', 'nova lox'],
                'ounces':[4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [164]:
meat_to_animal={
    'bacon':'pig'
    ,'pulled pork': 'pig'
    ,'pastrami': 'cow'
    ,'corned beef':'cow'
    ,'honey ham':'pig'
    ,'nova lox': 'salmon'
}

##### 소문자 만들기
그냥 str.lower()해도 되지만, 속도는 map(str.lower)이 더 빠르니 이걸 쓰자
1. 414us- %timeit(data['food'].map(str.lower).map(meat_to_animal))
2. 461us- %timeit(data['food'].str.lower().map(meat_to_animal))

In [172]:
# %timeit(data['food'].map(str.lower).map(meat_to_animal))

# %timeit(data['food'].str.lower().map(meat_to_animal))

##### 새 칼럼 만들 땐, Series도 괜찮나봄
아까 기존 칼럼 내용 바꿀땐, array만 되더니?

In [184]:
print(type(data['food'].map(str.lower)))
data['food'].map(str.lower)#.map(meat_to_animal)

<class 'pandas.core.series.Series'>


0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [186]:
data['animal']= data['food'].map(str.lower)#.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,bacon
1,pulled pork,3.0,pulled pork
2,bacon,12.0,bacon
3,Pastrami,6.0,pastrami
4,corned beef,7.5,corned beef
5,Bacon,8.0,bacon
6,pastrami,3.0,pastrami
7,honey ham,5.0,honey ham
8,nova lox,6.0,nova lox


In [187]:
#딕셔너리 키: 칼럼1에 대응
#딕셔너리 값: 새칼럼에 들어갈 내용
#새칼럼= Series_칼럼1.map(딕셔너리)
data['animal']= data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [189]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [191]:
data['animal']= data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [192]:
meat_to_animal['bacon']

'pig'

## 값 치환- replace
replace도 얕복. 
p275

In [194]:
#-999: 누락된 데이터라 치자 -> np.nan
data= Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [196]:
#값 하나를 치환
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

##### 여러 값을 한번에 치환- list

In [197]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [198]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

##### 값마다 다르게 치환
1. list
2. dictionary

In [199]:
data.replace([-999,-1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [200]:
data.replace({
    -999: np.nan
    ,-1000: 0
})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## 축색인 이름바꾸기
p277. 새 자료구조 생성없이, 바로 축이름 변경

In [201]:
data= DataFrame(np.arange(12).reshape((3,4))
               , index= ['Ohio','Colorado','New York']
               , columns= ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [202]:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [204]:
data.index= data.index.map(str.upper)

##### 얕은복사 columns

In [206]:
data.columns= data.columns.map(str.upper)

In [207]:
data

,ONE,TWO,THREE,FOUR
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


#### 깊은복사 rename -끔찍

In [208]:
data.rename(index= str.title, columns= str.lower)

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [209]:
data

,ONE,TWO,THREE,FOUR
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


##### 칼럼명 일부만 변경

In [210]:
data.rename(index= {'OHIO':'INDIANA'}
           ,columns= {'THREE':'peekaboo'})

,ONE,TWO,peekaboo,FOUR
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [211]:
data

,ONE,TWO,THREE,FOUR
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [213]:
data.heap()

AttributeError: 'DataFrame' object has no attribute 'heap'

##### 끔직한 깊은복사 rename 보완
inplace= True

In [212]:
_= data.rename(index= {'OHIO': 'INDIANA'}
              ,inplace= True)
data

,ONE,TWO,THREE,FOUR
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


## 개별화 , 양자화
p279

## 이상치 찾기/제외
p282

## 예제
1. sales데이터> 날짜별 판매량 합계
2. 각 code별 판매량 합계
3. product데이터> 1,2번 결과에서 총 매출 합계를 변경, 각 항목 이름 나타내기
4. emp데이터> 각 직원 연봉 기반, 각 직원별 등급 나타내는 그룹
    1. cut
    2. sign 함수 이용

| 연봉별 등급분리 체계   |
|---|---|
|  [0,2000)  |  C  |
|  [2000,3000) | B  |
|  [3000, | A  |

In [26]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [58]:
# %cd data
# %ls
# %cd ..

In [28]:
xls_file1= pd.ExcelFile('data/sales.xlsx')
xls_file1.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

In [59]:
sales= xls_file1.parse('Sheet1'); sales.head(10)
# Sheet2, 3은 공백
# sales1= xls_file1.parse('Sheet2'); sales1
# sales1= xls_file1.parse('Sheet3'); sales1

,date,code,qty
0,2018-01-01,c1,40
1,2018-01-01,c2,34
2,2018-01-01,c3,23
3,2018-01-01,c4,65
4,2018-01-02,c1,44
5,2018-01-02,c2,23
6,2018-01-02,c3,13
7,2018-01-02,c4,45
8,2018-01-03,c1,45
9,2018-01-03,c2,86


In [46]:
sales.columns

Index(['date', 'code', 'qty'], dtype='object')

### 1

In [60]:
# 다나옴 sales[sales.duplicated(['date'])]
# 다나옴 sales[sales['date'].duplicated()]
# 다나옴 sales.drop_duplicates
# 다나옴# sales_date= sales['date']
# sales_date.drop_duplicates

In [65]:
#안됨 sales.merge(sales, sales, left_on='date', right_on='qty')

In [81]:
date_qty= sales.groupby('date')[['qty']].sum()
date_qty

,qty
date,
2018-01-01,162
2018-01-02,125
2018-01-03,231
2018-01-04,168
2018-01-05,207
2018-01-06,167
2018-01-07,26


In [125]:
# 저번 강사님 코드 응용. 좀 뭔가..
f= lambda x: sales.qty[sales.date==x].sum()
sales['price']= sales.date.apply(f)

In [126]:
sales.head(2)

,date,code,qty,price
0,2018-01-01,c1,40,162
1,2018-01-01,c2,34,162


In [102]:
# 찜찜해서 일단 지움
# del sales['qty_sum']

### 강사님코드

In [161]:
sales.pivot('date','code','qty')#.sum(0)

code,c1,c2,c3,c4
date,,,,
2018-01-01,40,34,23,65
2018-01-02,44,23,13,45
2018-01-03,45,86,24,76
2018-01-04,45,24,76,23
2018-01-05,65,65,23,54
2018-01-06,56,76,23,12
2018-01-07,5,7,4,10


In [162]:
sales.pivot('date','code','qty').sum(0)

code
c1    300
c2    315
c3    186
c4    285
dtype: int64

### 2

In [70]:
sales.head(3)

,date,code,qty,qty_sum
0,2018-01-01,c1,40,162
1,2018-01-01,c2,34,162
2,2018-01-01,c3,23,162


In [80]:
code_qty= sales.groupby('code')[['qty']].sum()
code_qty

,qty
code,
c1,300
c2,315
c3,186
c4,285


In [77]:
xls_file1.close()

### 3

In [75]:
%cd data
%ls
%cd ..

E:\github\dataAnal_py_basic\popularPkg\data
 E 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 2468-0A05

 E:\github\dataAnal_py_basic\popularPkg\data 디렉터리

2018-03-13  오후 12:42    <DIR>          .
2018-03-13  오후 12:42    <DIR>          ..
2018-03-12  오후 03:51             8,722 dept.xlsx
2018-03-09  오후 02:43               308 disease.txt
2018-03-12  오후 03:46             9,063 emp_1.xlsx
2018-03-12  오후 03:46             8,050 emp_2.xlsx
2018-03-12  오후 03:49             8,046 emp_3.xlsx
2018-03-09  오전 10:17               268 ex_180308.txt
2018-03-07  오후 12:46                35 ex_df.txt
2018-03-07  오전 11:09                20 ex_text.txt
2018-03-12  오후 06:31           229,849 kgg_interview.csv
2018-03-13  오후 12:26             8,564 product.xlsx
2018-03-13  오후 12:23            11,007 sales.xlsx
2018-03-07  오전 11:05                20 save_test2
              12개 파일             283,952 바이트
               2개 디렉터리  226,282,508,288 바이트 남음
E:\github\dataAnal_py_basic\popularPkg


In [78]:
xls_file2= pd.ExcelFile('data/product.xlsx')
xls_file2.sheet_names

['Sheet1']

In [79]:
product= xls_file2.parse('Sheet1'); product

,code,product,price
0,c1,notebook,249
1,c2,tv,149
2,c3,refrigerator,300
3,c4,washer,120


In [83]:
date_qty

,qty
date,
2018-01-01,162
2018-01-02,125
2018-01-03,231
2018-01-04,168
2018-01-05,207
2018-01-06,167
2018-01-07,26


In [84]:
code_qty

,qty
code,
c1,300
c2,315
c3,186
c4,285


In [92]:
code_qty.shape

(4, 1)

In [96]:
type(code_qty.qty)

pandas.core.series.Series

In [86]:
code_qty.columns

Index(['qty'], dtype='object')

In [87]:
code_qty.qty

code
c1    300
c2    315
c3    186
c4    285
Name: qty, dtype: int64

In [88]:
pd.Series(code_qty.qty)

code
c1    300
c2    315
c3    186
c4    285
Name: qty, dtype: int64

##### array로 변환해야 dataframe에 들어간다
Series는 안 됨

In [95]:
code_qty.values

array([[300],
       [315],
       [186],
       [285]], dtype=int64)

In [85]:
#안됨
product.price= code_qty.qty
product

,code,product,price
0,c1,notebook,NaN
1,c2,tv,NaN
2,c3,refrigerator,NaN
3,c4,washer,NaN


In [91]:
#안됨
product.replace('price', code_qty.qty)
product

,code,product,price
0,c1,notebook,NaN
1,c2,tv,NaN
2,c3,refrigerator,NaN
3,c4,washer,NaN


In [94]:
#array만 들어가더라
product.price= code_qty.values
product

,code,product,price
0,c1,notebook,300
1,c2,tv,315
2,c3,refrigerator,186
3,c4,washer,285


##### 각 항목의 이름 ?

### 4
emp데이터> 각 직원 연봉 기반, 각 직원별 등급 나타내는 그룹
1. cut
2. sign 함수 이용

In [104]:
!start .

In [107]:
xls_file= pd.ExcelFile('data/emp_1.xlsx')
emp1= xls_file.parse(xls_file.sheet_names[0])
emp1.head()

,EMPNO,ENAME,JOB
0,7369,SMITH,CLERK
1,7499,ALLEN,SALESMAN
2,7521,WARD,SALESMAN
3,7566,JONES,MANAGER
4,7654,MARTIN,SALESMAN


In [108]:
xls_file= pd.ExcelFile('data/emp_2.xlsx')
emp2= xls_file.parse(xls_file.sheet_names[0])
emp2.head()

,EMPNO,SAL,COMM,DEPTNO
0,7369,800,NaN,20
1,7499,1600,300.0,30
2,7521,1250,500.0,30
3,7566,2975,NaN,20
4,7654,1250,1400.0,30


In [109]:
xls_file= pd.ExcelFile('data/emp_3.xlsx')
emp3= xls_file.parse(xls_file.sheet_names[0])
emp3.head()

,EMPNO,ENAME,JOB,SAL,COMM,DEPTNO
0,9999,PARK,CLERK,5000,100.0,10
1,9998,KIM,MANAGER,4800,20.0,20
2,9997,YOO,ANALYST,6000,500.0,30
3,9996,JUNG,CLERK,5100,NaN,10
4,9995,HYUN,MANAGER,4900,NaN,20


In [120]:
emp123= pd.merge(pd.merge(emp1, emp2, on='EMPNO'), emp3, how='outer')
emp123.sort_values('SAL', ascending=False)

,EMPNO,ENAME,JOB,SAL,COMM,DEPTNO
19,9994,CHOI,ANALYST,6100,NaN,30
16,9997,YOO,ANALYST,6000,500.0,30
17,9996,JUNG,CLERK,5100,NaN,10
14,9999,PARK,CLERK,5000,100.0,10
8,7839,KING,PRESIDENT,5000,NaN,10
18,9995,HYUN,MANAGER,4900,NaN,20
15,9998,KIM,MANAGER,4800,20.0,20
7,7788,SCOTT,ANALYST,3000,NaN,20
12,7902,FORD,ANALYST,3000,NaN,20
3,7566,JONES,MANAGER,2975,NaN,20


In [121]:
pd.cut(emp123, [2000,3000,6101], right=False)

TypeError: '<' not supported between instances of 'int' and 'str'

# 강사님 코드

### 3

In [150]:
sales.columns

Index(['date', 'code', 'qty', 'price', 'sal_tot'], dtype='object')

In [159]:
sales['sal_tot']= sales.qty* sales.price
sales.head(3)

,date,code,qty,price,sal_tot
0,2018-01-01,c1,40,162,6480
1,2018-01-01,c2,34,162,5508
2,2018-01-01,c3,23,162,3726


In [147]:
sales['sal_tot']= sales[['qty','price']].product(axis=1)
sales.head(3)

,date,code,qty,price,sal_tot
0,2018-01-01,c1,40,162,6480
1,2018-01-01,c2,34,162,5508
2,2018-01-01,c3,23,162,3726


In [151]:
sales.pivot('date', 'price', 'sal_tot').sum(1)

ValueError: Index contains duplicate entries, cannot reshape

### 4

In [137]:
# right= False: [이상,미만)
pd.cut(emp123.SAL, [0,2000,3000,6101], right= False)

0        [0, 2000)
1        [0, 2000)
2        [0, 2000)
3     [2000, 3000)
4        [0, 2000)
5     [2000, 3000)
6     [2000, 3000)
7     [3000, 6101)
8     [3000, 6101)
9        [0, 2000)
10       [0, 2000)
11       [0, 2000)
12    [3000, 6101)
13       [0, 2000)
14    [3000, 6101)
15    [3000, 6101)
16    [3000, 6101)
17    [3000, 6101)
18    [3000, 6101)
19    [3000, 6101)
Name: SAL, dtype: category
Categories (3, interval[int64]): [[0, 2000) < [2000, 3000) < [3000, 6101)]

In [138]:
pd.cut(emp123.SAL, [0,2000,3000,6101], right= False, labels=['C','B','A'])

0     C
1     C
2     C
3     B
4     C
5     B
6     B
7     A
8     A
9     C
10    C
11    C
12    A
13    C
14    A
15    A
16    A
17    A
18    A
19    A
Name: SAL, dtype: category
Categories (3, object): [C < B < A]

In [152]:
print(np.sign(-10))
print(np.sign(0))
print(np.sign(20))

-1
0
1


# ????

# ????

# ????

# ????

# ????

#### 조건문 최소화
[0, 2000) < [2000, 3000) < [3000, 6101) 에서
1. .sign문을 3개 더하면 2000보다 작은 건 -3이 될 것
> ??? 

In [157]:
grade_sign= np.sign(emp123.SAL-2000)+ np.sign(emp123.SAL-3000)+ np.sign(emp123.SAL-6101)
emp123['grade']= grade_sign.replace([-3,-1,0,1],['D','C','B','A'])

In [158]:
emp123

,EMPNO,ENAME,JOB,SAL,COMM,DEPTNO,grade
0,7369,SMITH,CLERK,800,NaN,20,D
1,7499,ALLEN,SALESMAN,1600,300.0,30,D
2,7521,WARD,SALESMAN,1250,500.0,30,D
3,7566,JONES,MANAGER,2975,NaN,20,C
4,7654,MARTIN,SALESMAN,1250,1400.0,30,D
5,7698,BLAKE,MANAGER,2850,NaN,30,C
6,7782,CLARK,MANAGER,2450,NaN,10,C
7,7788,SCOTT,ANALYST,3000,NaN,20,B
8,7839,KING,PRESIDENT,5000,NaN,10,A
9,7844,TURNER,SALESMAN,1500,0.0,30,D


In [142]:
np.sign(emp123.SAL- 2000)

0    -1
1    -1
2    -1
3     1
4    -1
5     1
6     1
7     1
8     1
9    -1
10   -1
11   -1
12    1
13   -1
14    1
15    1
16    1
17    1
18    1
19    1
Name: SAL, dtype: int64

In [143]:
np.sign(emp123.SAL- 3000)

0    -1
1    -1
2    -1
3    -1
4    -1
5    -1
6    -1
7     0
8     1
9    -1
10   -1
11   -1
12    0
13   -1
14    1
15    1
16    1
17    1
18    1
19    1
Name: SAL, dtype: int64